In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-upda

In [26]:
# JAVA_HOME 환경변수 설정하기
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [27]:
import pandas as pd
import numpy as np
import random
import re, string
from konlpy.tag import Hannanum
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [28]:
hannanum = Hannanum()

In [29]:
train = pd.read_excel('/content/drive/MyDrive/temp/Newsletter_Board/데이터_2022.05.18.xlsx')
train2 = train.copy()

In [30]:
train.head()

,Title,url,thumbnail
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...


## 전처리
### 정제 및 정규화
코퍼스 내 기호 및 글자들을 일정 규칙에 기반하여 텍스트 정제

### 불용어 제거
한글은 따로 불용어 사전이 존재하지 않기에 불용어 사전을 직접 정의해줘야 함 -> 일반적으로 접속사나 조사 등을 불용어로 정의  
기본 불용어는 https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt 를 참고하고, 이후 단어 빈도수를 확인한 뒤에 불용어를 추가 정의해주는 방향

In [31]:
with open('/content/drive/MyDrive/temp/Newsletter_Board/stopwords.txt', 'r', encoding='cp949') as f:
  list_file = f.readlines() 
stopwords = list_file[0].split(",")

In [41]:
# 정규화
def preprocess(text):
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

# 한글만 추출
# def extract_word(text):
#     hangul = re.compile('[^가-힣]') 
#     result = hangul.sub(' ', text) 
#     return result

# 형태소 반환
def morpheme(text):
  m = hannanum.morphs(text)
  return m

# 품사
def pos(text):
  p = hannanum.pos(text)
  return p

# 명사 추출 / 한 글자 제외 / 불용어 제거
def noun(text):
  n = []
  word = hannanum.nouns(text)
  for w in word:
    if len(w)>1 and w not in stopwords:
      n.append(w)  
  # return " ".join(n)
  return n

# 정규화 후 명사 추출
def finalpreprocess(text):
  return noun(extract_word(text))

In [42]:
train['normal'] = train['Title'].apply(lambda x: preprocess(x))

In [ ]:
# 명사 리스트는 전체 단위 키워드 추출 위해 정의

train['noun'] = train['Title'].apply(lambda x: " ".join(finalpreprocess(x)))
train['noun_list'] = train['Title'].apply(lambda x: finalpreprocess(x))

In [43]:
train

,Title,url,thumbnail,normal
0,'과학방역' 내세운 중대본…전문가위원회·빅데이터 플랫폼 구성,https://www.yna.co.kr/view/AKR2022051803830053...,https://search.pstatic.net/common/?src=https%3...,과학방역 내세운 중대본전문가위원회빅데이터 플랫폼 구성
1,"넥슨, 라프텔과 '가명정보 결합'…데이터 사이언스 강화",http://www.newsis.com/view/?id=NISX20220518_00...,https://search.pstatic.net/common/?src=https%3...,넥슨 라프텔과 가명정보 결합 데이터 사이언스 강화
2,"인공위성 '세종 1호' 쏘는 한글과컴퓨터, 이미지 데이터 노린다",http://www.bloter.net/newsView/blt202205180027,https://search.pstatic.net/common/?src=https%3...,인공위성 세종 호 쏘는 한글과컴퓨터 이미지 데이터 노린다
3,"넥슨, 라프텔과 가명정보 결합…""데이터 사이언스 강화""",https://www.news1.kr/articles/?4683961,https://search.pstatic.net/common/?src=https%3...,넥슨 라프텔과 가명정보 결합 데이터 사이언스 강화
4,"KT SAT ""스페이스데이터 서비스 진출…글로벌 브랜드 키운다""",https://www.hankyung.com/it/article/202205184568i,https://search.pstatic.net/common/?src=https%3...,KT SAT 스페이스데이터 서비스 진출글로벌 브랜드 키운다
...,...,...,...,...
2616,﻿[2022지방선거] ﻿1400만 경기도민 표심 잡아라…경기도지사 후보 김동연 vs...,http://www.smartfn.co.kr/view.php?ud=202205181...,https://search.pstatic.net/common/?src=https%3...,지방선거 만 경기도민 표심 잡아라경기도지사 후보 김동연 vs 김은혜
2617,"[사설] 가팔라질 기준금리 인상, 부채위기 확산 막아야",https://www.etoday.co.kr/news/view/2134715,,사설 가팔라질 기준금리 인상 부채위기 확산 막아야
2618,﻿[2022지방선거] 세종시장 후보 이춘희 “행정수도 완성” vs 최민호 “윤석열 ...,http://www.smartfn.co.kr/view.php?ud=202205181...,https://search.pstatic.net/common/?src=https%3...,지방선거 세종시장 후보 이춘희 행정수도 완성 vs 최민호 윤석열 정부 새 시작
2619,"뉴욕증시 비트코인 흔들 제롬파월 연설 ""자이언트 빅스텝"" 소매판매 0.9%",http://www.g-enews.com/ko-kr/news/article/news...,https://search.pstatic.net/common/?src=https%3...,뉴욕증시 비트코인 흔들 제롬파월 연설 자이언트 빅스텝 소매판매


## 키워드 추출
(만약 군집화 전 키워드를 추출한다면 어떨까 싶어.. 한번 해봤습니다..ㅎㅎ)

In [ ]:
words = []
for word_list in train['noun_list']:
  for word in word_list:
    words.append(word)

In [ ]:
# 상위 n개 추출
n = 20
Counter(words).most_common(n)

[('선정', 189),
 ('디지털', 117),
 ('사업', 104),
 ('스마트시티', 99),
 ('부산', 95),
 ('구축', 94),
 ('플랫폼', 89),
 ('데이터', 84),
 ('국내', 84),
 ('기업', 75),
 ('개최', 74),
 ('빅데이터', 71),
 ('서비스', 70),
 ('개발', 64),
 ('정부', 63),
 ('솔루션', 60),
 ('우선협상대상자', 59),
 ('추진', 58),
 ('기술', 55),
 ('출범', 54)]

### 키워드 관련 논의 사항
- 데이터 자체는 우리 주제이므로 불용어 사전에 추가 필요
- 빅데이터 추가 여부
- 선정 / 부산 / 개최 등의 키워드가 유의미한가

## 벡터화

- TF-IDF
- CountVectorizer

In [ ]:
corpus = train['noun']

In [ ]:
# TF-IDF

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'과학방역': 346, '중대본': 3323, '전문가위원회': 3082, '빅데이터': 1611, '플랫폼': 3998, '구성': 417, '넥슨': 705, '라프텔': 1042, '가명정보': 10, '결합': 175, '데이터': 909, '사이언스': 1673, '강화': 75, '인공위성': 2835, '세종': 1892, '한글과컴퓨터': 4083, '이미지': 2780, '스페이스데이터': 2063, '서비스': 1791, '진출': 3422, '글로벌': 513, '브랜드': 1564, '농협은행': 754, '마이데이터': 1117, '친구추천': 3642, '이벤트': 2783, '진행': 3423, '하동군': 4025, '공공데이터': 288, '기업매칭': 590, '공모': 308, '선정': 1833, '행안부': 4178, '공기관': 295, '포함': 3954, '기관': 557, '데이터기반행정': 911, '점검': 3124, '교통사망사고': 402, '도민생명': 931, '랜섬웨': 1044, '기업': 587, '지불': 3367, '복구': 1487, '실패': 2196, '안산시': 2264, '지원사업': 3384, '추진': 3581, '질병청장': 3428, '감염병': 54, '컨트롤타워': 3689, '재정립': 3003, '활용': 4332, '과학': 341, '방역할것': 1384, '인천관광공사': 2868, '기업매칭사업': 591, '국비': 447, '보령시': 1459, '연속': 2442, '정부': 3152, '구축': 423, '교육': 392, '돌봄시설': 952, '공기정화': 297, '지원': 33

In [ ]:
# CountVectorizer

vect = CountVectorizer().fit(corpus)
print(vect.transform(corpus).toarray())
print(vect.vocabulary_)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'과학방역': 346, '중대본': 3323, '전문가위원회': 3082, '빅데이터': 1611, '플랫폼': 3998, '구성': 417, '넥슨': 705, '라프텔': 1042, '가명정보': 10, '결합': 175, '데이터': 909, '사이언스': 1673, '강화': 75, '인공위성': 2835, '세종': 1892, '한글과컴퓨터': 4083, '이미지': 2780, '스페이스데이터': 2063, '서비스': 1791, '진출': 3422, '글로벌': 513, '브랜드': 1564, '농협은행': 754, '마이데이터': 1117, '친구추천': 3642, '이벤트': 2783, '진행': 3423, '하동군': 4025, '공공데이터': 288, '기업매칭': 590, '공모': 308, '선정': 1833, '행안부': 4178, '공기관': 295, '포함': 3954, '기관': 557, '데이터기반행정': 911, '점검': 3124, '교통사망사고': 402, '도민생명': 931, '랜섬웨': 1044, '기업': 587, '지불': 3367, '복구': 1487, '실패': 2196, '안산시': 2264, '지원사업': 3384, '추진': 3581, '질병청장': 3428, '감염병': 54, '컨트롤타워': 3689, '재정립': 3003, '활용': 4332, '과학': 341, '방역할것': 1384, '인천관광공사': 2868, '기업매칭사업': 591, '국비': 447, '보령시': 1459, '연속': 2442, '정부': 3152, '구축': 423, '교육': 392, '돌봄시설': 952, '공기정화': 297, '지원': 3382, '종합': 3249, '한국축산데이터': 4078, '축산

## 테스트

In [ ]:
sample_df = train2.sample(n=20, random_state=520)

In [ ]:
sample_df

,Title,url,thumbnail
15,인천관광공사 공공데이터 기업매칭사업 국비 10억,https://news.kmib.co.kr/article/view.asp?arcid...,https://search.pstatic.net/common/?src=https%3...
1363,"국토부, 부동산서비스협의체 출범…'프롭테크+중개업소' 상생",https://biz.newdaily.co.kr/site/data/html/2022...,https://search.pstatic.net/common/?src=https%3...
308,"""과거 사진은 찾았는데 친구들은..."" 싸이월드, SNS 기능 상실했나",http://www.edaily.co.kr/news/newspath.asp?news...,https://search.pstatic.net/common/?src=https%3...
85,'항공기 옆으로 쌩' UFO 포착…美의회 청문회서 첫 공개,http://news.mt.co.kr/mtview.php?no=20220518133...,https://search.pstatic.net/common/?src=https%3...
1692,"HLB테라퓨틱스, 주가 훨훨…항암치료제 호조에 급등",http://www.pinpointnews.co.kr/news/articleView...,https://search.pstatic.net/common/?src=https%3...
1080,"120년 한국 잡지 창간호, 세상의 빛을 본다",http://www.whosaeng.com/136163,https://search.pstatic.net/common/?src=https%3...
371,"넷플릭스 망 무임승차 재판, 핵심쟁점으로 ‘무정산’ 부각",https://zdnet.co.kr/view/?no=20220518195751,https://search.pstatic.net/common/?src=https%3...
716,"""인수價 못깎아줘""…못 박은 트위터",https://www.hankyung.com/international/article...,
2595,"신한베트남은행, 디지털 사업 전담 추진 조직 'Future Bank Group' 출범",https://www.onews.tv/news/articleView.html?idx...,https://search.pstatic.net/common/?src=https%3...
1599,"""유병언 사망 시점, 구더기는 알았다""...경찰, '법곤충감정실' 본격 가동",https://science.ytn.co.kr/program/program_view...,https://search.pstatic.net/common/?src=https%3...


In [ ]:
sample_df['noun'] = sample_df['Title'].apply(lambda x: " ".join(finalpreprocess(x)))
sample_df

,Title,url,thumbnail,noun
15,인천관광공사 공공데이터 기업매칭사업 국비 10억,https://news.kmib.co.kr/article/view.asp?arcid...,https://search.pstatic.net/common/?src=https%3...,인천관광공사 공공데이터 기업매칭사업 국비
1363,"국토부, 부동산서비스협의체 출범…'프롭테크+중개업소' 상생",https://biz.newdaily.co.kr/site/data/html/2022...,https://search.pstatic.net/common/?src=https%3...,국토부 부동산서비스협의체 출범 프롭테크 중개업소 상생
308,"""과거 사진은 찾았는데 친구들은..."" 싸이월드, SNS 기능 상실했나",http://www.edaily.co.kr/news/newspath.asp?news...,https://search.pstatic.net/common/?src=https%3...,과거 사진 친구들 싸이월드 기능 상실
85,'항공기 옆으로 쌩' UFO 포착…美의회 청문회서 첫 공개,http://news.mt.co.kr/mtview.php?no=20220518133...,https://search.pstatic.net/common/?src=https%3...,항공기 포착 의회 청문회 공개
1692,"HLB테라퓨틱스, 주가 훨훨…항암치료제 호조에 급등",http://www.pinpointnews.co.kr/news/articleView...,https://search.pstatic.net/common/?src=https%3...,테라퓨틱스 항암치료제 호조 급등
1080,"120년 한국 잡지 창간호, 세상의 빛을 본다",http://www.whosaeng.com/136163,https://search.pstatic.net/common/?src=https%3...,한국 창간호 세상
371,"넷플릭스 망 무임승차 재판, 핵심쟁점으로 ‘무정산’ 부각",https://zdnet.co.kr/view/?no=20220518195751,https://search.pstatic.net/common/?src=https%3...,넷플릭스 무임승차 재판 핵심쟁점 무정산 부각
716,"""인수價 못깎아줘""…못 박은 트위터",https://www.hankyung.com/international/article...,,못깎아줘 트위터
2595,"신한베트남은행, 디지털 사업 전담 추진 조직 'Future Bank Group' 출범",https://www.onews.tv/news/articleView.html?idx...,https://search.pstatic.net/common/?src=https%3...,신한베트남은행 디지털 사업 전담 추진 조직 출범
1599,"""유병언 사망 시점, 구더기는 알았다""...경찰, '법곤충감정실' 본격 가동",https://science.ytn.co.kr/program/program_view...,https://search.pstatic.net/common/?src=https%3...,유병언 사망 시점 구더기 경찰 법곤충감정실 가동


In [ ]:
tfidfv = TfidfVectorizer().fit(sample_df['noun'])
print(tfidfv.transform(sample_df['noun']).toarray())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.3732781  ... 0.         0.42465484 0.        ]]


In [ ]:
vect = CountVectorizer().fit(sample_df['noun'])
print(vect.transform(sample_df['noun']).toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 1 0]]
